In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image
from sklearn.metrics import classification_report,confusion_matrix
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D
# from tensorflow.keras.applications import EfficientNetB0
from keras.optimizers import Adam,RMSprop

from keras.applications.resnet50 import ResNet50
from keras.layers import Dense,Dropout,Activation,Flatten
from keras import Model
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# import cv2
import os
import itertools 
import pandas as pd
import random

In [ ]:
DATADIR  =  'COVID-19_Radiography_Dataset/'

In [ ]:
#split de datagen into 70% and 30%, train and test
train = ImageDataGenerator(rescale=1./255)
validation = ImageDataGenerator(rescale=1./255, shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

classes=["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]
# classes=["COVID",  "Normal"]


In [ ]:
num_classes = len(classes)
# print(num_classes)
batch_size=32


#split de datagen into 70% and 30%, train and test
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,\
                             height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, \
                             fill_mode="nearest", rescale=1./255, validation_split=0.3)


#load the images to training

train_gen = datagen.flow_from_directory(directory=DATADIR, 
                                              target_size=(200, 200),
                                              class_mode='categorical',
                                              subset='training',
                                              shuffle=True, classes=classes,
                                              batch_size=batch_size, 
                                              color_mode="grayscale")
#load the images to test
test_gen = datagen.flow_from_directory(directory=DATADIR, 
                                              target_size=(200, 200),
                                              class_mode='categorical',
                                              subset='validation',
                                              shuffle=True, classes=classes,
                                              batch_size=batch_size, 
                                              color_mode="grayscale")

In [ ]:
train_gen.class_indices

In [ ]:
train_gen.classes

#### We are building a simple Neural Network for Classification

In [ ]:
model =  tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
#     
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=["accuracy"],optimizer=RMSprop(lr=0.03))

In [ ]:
model.summary()

In [ ]:
model_fit  = model.fit(train_gen,steps_per_epoch=3, epochs=30,validation_data=test_gen)

### We are taking a look at some the test images

In [ ]:
N = 40
int(np.floor(N/4))

In [ ]:
def sample_Databounch(DATADIR,N=20):
    plt.figure(figsize=(15,14))
    secure_random = random.SystemRandom()
    k = 0
    m = 4
    n  =  int(np.floor(N/m))
    N  =  m * n
    for i in range(N):
            path  =  os.path.join(DATADIR,secure_random.choice(test_gen.filenames))
            if(path.endswith('.png')):
                k =  k + 1
                plt.subplot(m,n,k)
                img  = image.load_img(path,target_size=(200,200,3))
                plt.imshow(img)
                plt.title(os.path.splitext(os.path.basename(path))[0])

sample_Databounch(DATADIR,N=20)

In [ ]:
plt.plot(model_fit.history['loss'])
plt.plot(model_fit.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(model_fit.history['accuracy'])
plt.plot(model_fit.history['val_accuracy'])
plt.title('Model  accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
img_height =  200
img_width  =  200
n_channels = 1
base_model = ResNet50(weights= None, include_top=False, \
                                            input_shape= (img_height,img_width,n_channels))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(len(classes), activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)